In [28]:
import numpy as np
import sys
import os
#sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import glob
import cv2
import matplotlib.pyplot as plt
import pandas
import imutils
from tqdm import tqdm
from cv2 import VideoWriter
global rect
from PIL import Image as im
import math

#1st vehicle - RED
v1_lower_bound = np.array([100, 15, 15])
v1_upper_bound = np.array([240, 120, 120])
#2nd vehicle - BLUE
v2_lower_bound = np.array([5, 5, 100])
v2_upper_bound = np.array([70, 170, 255])
#3rd vehicle - GREEN
v3_lower_bound = np.array([10, 110, 20])
v3_upper_bound = np.array([90, 250, 180])

#BGR colors
bgr_red = (50, 50, 255)
bgr_blue = (255, 175, 70)
bgr_green = (0, 255, 0)


#variables to compute velocity, acceleration
prev_ftime = "0"

v1_x_prev_ref = 0
v1_y_prev_ref = 0
v1_prev_velocity = 0.0

v2_x_prev_ref = 0
v2_y_prev_ref = 0
v2_prev_velocity = 0.0

v3_x_prev_ref = 0
v3_y_prev_ref = 0
v3_prev_velocity = 0.0

#conversion - 60 pixel = 28 cm
pixel_to_meter = 28 / 60 / 100


IMAGE_H = 720
IMAGE_W = 1280
dst = np.float32([[50, IMAGE_H+50], [IMAGE_W+50, IMAGE_H+50], [50, 50], [IMAGE_W+50, 50]])
src = np.float32([[85, 627], [1070, 636], [425, 165], [835, 162]])
M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix

fourcc = cv2.VideoWriter_fourcc(*'FMP4')
width = 1280
height = 720
FPS = 1 # was 20
seconds = 100
video = VideoWriter('video_FOV.avi', fourcc, float(FPS), (width, height))

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized
    

def get_coordinates(img, interval, frameDelta, prev_velocity, x_prev_ref, y_prev_ref, x_prev, y_prev, x_ref, y_ref, lower_bound, upper_bound, color, color_code, text_displacement, img_overlay):   
    final_x = 0;
    final_y = 0;
    final_yaw = 0;

    mask = cv2.inRange(frameDelta, lower_bound, upper_bound)
    thresh = cv2.threshold(mask, 30, 260, cv2.THRESH_BINARY)[1]
    
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cnts = imutils.grab_contours(cnts)
    
    # loop over the contours
    for c in cnts:
        print('c')
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 300:
            continue
        if cv2.contourArea(c) > 1000:
            continue
        # compute the bounding box for the contour, draw it on the frame,
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        
        x = int(rect[0][0])
        y = int(rect[0][1]) 
        
        horizontal = np.array([1,0])
        p0 = box[0]
        p1 = box[1] - box[0]
        p2 = box[2] - box[0]
        p3 = box[3] - box[0]
        
        sum1 = np.sum(p1**2)
        sum2 = np.sum(p2**2)
        sum3 = np.sum(p3**2)
        
        length1 = np.sqrt(sum1)
        length2 = np.sqrt(sum2)
        length3 = np.sqrt(sum3)
        
        list123 = [length1, length2, length3]
        mid_length = np.median(list123)
        
        if mid_length == length1:
            if y > 360:
                if box[1][0] > box[0][0]:
                    yaw_axis = p1
                    p_start = box[0]
                    p_end = box[1]
                elif box[1][0] < box[0][0]:
                    yaw_axis = box[0] - box[1]
                    p_start = box[1]
                    p_end = box[0]
                else:
                    if x > 640:
                        if box[1][1] > box[0][1]:
                            yaw_axis = box[0] - box[1]
                            p_start = box[1]
                            p_end = box[0]
                        elif box[1][1] < box[0][1]:
                            yaw_axis = p1
                            p_start = box[0]
                            p_end = box[1]
                    else:
                        if box[1][1] > box[0][1]:
                            yaw_axis = p1
                            p_start = box[0]
                            p_end = box[1]
                        elif box[1][1] < box[0][1]:
                            yaw_axis = box[0] - box[1]
                            p_start = box[1]
                            p_end = box[0]
            else:
                if box[1][0] > box[0][0]:
                    yaw_axis = box[0] - box[1]
                    p_start = box[1]
                    p_end = box[0]
                elif box[1][0] < box[0][0]:
                    yaw_axis = p1
                    p_start = box[0]
                    p_end = box[1]
                else:
                    if x > 640:
                        if box[1][1] > box[0][1]:
                            yaw_axis = box[0] - box[1]
                            p_start = box[1]
                            p_end = box[0]
                        elif box[1][1] < box[0][1]:
                            yaw_axis = p1
                            p_start = box[0]
                            p_end = box[1]
                    else:
                        if box[1][1] > box[0][1]:
                            yaw_axis = p1
                            p_start = box[0]
                            p_end = box[1]
                        elif box[1][1] < box[0][1]:
                            yaw_axis = box[0] - box[1]
                            p_start = box[1]
                            p_end = box[0]
                
        elif mid_length == length2:
            if y > 360:
                if box[2][0] > box[0][0]:
                    yaw_axis = p2
                    p_start = box[0]
                    p_end = box[2]
                elif box[2][0] < box[0][0]:
                    yaw_axis = box[0] - box[2]
                    p_start = box[2]
                    p_end = box[0]
                else:
                    if x > 640:
                        if box[2][1] > box[0][1]:
                            yaw_axis = box[0] - box[2]
                            p_start = box[2]
                            p_end = box[0]
                        elif box[2][1] < box[0][1]:
                            yaw_axis = p2
                            p_start = box[0]
                            p_end = box[2]
                    else:
                        if box[2][1] > box[0][1]:
                            yaw_axis = p2
                            p_start = box[0]
                            p_end = box[2]
                        elif box[2][1] < box[0][1]:
                            yaw_axis = box[0] - box[2]
                            p_start = box[2]
                            p_end = box[0]
            else:
                if box[2][0] > box[0][0]:
                    yaw_axis = box[0] - box[2]
                    p_start = box[2]
                    p_end = box[0]
                elif box[2][0] < box[0][0]:
                    yaw_axis = p2
                    p_start = box[0]
                    p_end = box[2]
                else:
                    if x > 640:
                        if box[2][1] > box[0][1]:
                            yaw_axis = box[0] - box[2]
                            p_start = box[2]
                            p_end = box[0]
                        elif box[2][1] < box[0][1]:
                            yaw_axis = p2
                            p_start = box[0]
                            p_end = box[2]
                    else:
                        if box[2][1] > box[0][1]:
                            yaw_axis = p2
                            p_start = box[0]
                            p_end = box[2]
                        elif box[2][1] < box[0][1]:
                            yaw_axis = box[0] - box[2]
                            p_start = box[2]
                            p_end = box[0]
                            
        elif mid_length == length3:
            if y > 360:
                if box[3][0] > box[0][0]:
                    yaw_axis = p3
                    p_start = box[0]
                    p_end = box[3]
                elif box[3][0] < box[0][0]:
                    yaw_axis = box[0] - box[3]
                    p_start = box[3]
                    p_end = box[0]
                else:
                    if x > 640:
                        if box[3][1] > box[0][1]:
                            yaw_axis = box[0] - box[3]
                            p_start = box[3]
                            p_end = box[0]
                        elif box[3][1] < box[0][1]:
                            yaw_axis = p3
                            p_start = box[0]
                            p_end = box[3]
                    else:
                        if box[3][1] > box[0][1]:
                            yaw_axis = p3
                            p_start = box[0]
                            p_end = box[3]
                        elif box[3][1] < box[0][1]:
                            yaw_axis = box[0] - box[3]
                            p_start = box[3]
                            p_end = box[0]
            else:
                if box[3][0] > box[0][0]:
                    yaw_axis = box[0] - box[3]
                    p_start = box[3]
                    p_end = box[0]
                elif box[3][0] < box[0][0]:
                    yaw_axis = p3
                    p_start = box[0]
                    p_end = box[3]
                else:
                    if x > 640:
                        if box[3][1] > box[0][1]:
                            yaw_axis = box[0] - box[3]
                            p_start = box[3]
                            p_end = box[0]
                        elif box[3][1] < box[0][1]:
                            yaw_axis = p3
                            p_start = box[0]
                            p_end = box[3]
                    else:
                        if box[3][1] > box[0][1]:
                            yaw_axis = p3
                            p_start = box[0]
                            p_end = box[3]
                        elif box[3][1] < box[0][1]:
                            yaw_axis = box[0] - box[3]
                            p_start = box[3]
                            p_end = box[0]
                
        angle = np.arctan2(yaw_axis[1], yaw_axis[0])
        yaw = -int(np.degrees(angle))     
        if yaw < 0:
            yaw += 360
        
        cv2.drawContours(img, [box], 0, (0, 255, 0), 3)  
        cv2.arrowedLine(img, ((p_start[0]), (p_start[1])), (p_end[0], p_end[1]), (0, 0, 255), 3)
        
        
        cv2.putText(img, "{}".format(color), (70 + text_displacement, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_code, 2)
        text = "[Center: (" + str('{:.1f}'.format((x-x_ref)*pixel_to_meter)) + "m," + str('{:.1f}'.format((y_ref-y)*pixel_to_meter)) + "m)]"        
        cv2.putText(img, "{}".format(text), (70 + text_displacement, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_code, 2)
        text = "[Yaw: " +  str(yaw) + " degree]"        
        cv2.putText(img, "{}".format(text), (70 + text_displacement, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_code, 2)
        final_x = int(rect[0][0])
        final_y = int(rect[0][1])
        final_yaw = yaw
        
        #reference frame is printed on every function call - need to optimize
        text = "Reference Frame"
        cv2.putText(img, "{}".format(text), (x_ref+50, y_ref+30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.line(img, (x_ref,y_ref), (x_ref, y_ref-20), (0,255,0), 3)
        cv2.line(img, (x_ref,y_ref), (x_ref+50, y_ref), (0,255,0), 3)
        text = "0 degree"
        cv2.putText(img, "{}".format(text), (x_ref+60, y_ref+5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        
        #draw yellow ellipse
        cv2.ellipse(img_overlay, (x, y), (75,50), -yaw, -90, 90, (0, 255, 255), -1)        

        #finding distance with pythagorean theorem
        distance = math.sqrt(((x-x_ref-x_prev_ref)**2) + ((y_ref-y-y_prev_ref)**2))
        velocity = ((distance * pixel_to_meter) / interval)
        text = "[Velo.: " + str('{:.1f}'.format(velocity*3600/1000)) + " km/h]"        
        cv2.putText(img, "{}".format(text), (70 + text_displacement, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_code, 2)  
        #interval of three frames for acceleration
        #interval = float(ftime) - float(prev_ftime)
        acceleration = (((velocity - prev_velocity)) / (interval))
        
        text = "[Acc.: " + str('{:+.1f}'.format(acceleration)) + " m/s^2]"
        cv2.putText(img, "{}".format(text), (70 + text_displacement, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_code, 2)
        
        #set variable values to compute for next iteration
        x_prev_ref = (x-x_ref)
        y_prev_ref = (y_ref-y)
        prev_velocity = velocity
        break
    
    # Return the coordinates
    return final_x, final_y, final_yaw, img, prev_velocity, x_prev_ref, y_prev_ref, img_overlay

def get_interval(ftime):
    
    global prev_ftime
    #instead of using constant value 17, use space as buffer for substring
    ftime = ftime[17:-1]
    #inverval of two frames for velocity
    print(ftime)
    interval = float(ftime) - float(prev_ftime)
    prev_ftime = ftime
    return interval
    
def get_ref_coordinates(background_path):
    #get reference frame coordinates based on bottom-most and left-most cone 
    img = im.open(background_path)
    w, h = img.size
    pix = img.load()
    x_ref, y_ref = 0, 0
    #iterables chosen such that loop only covers bottom left section of background image
    for i in range(int(3*h/4),h):
        for j in range(int(w/4)):
            r, g, b, *A = pix[j,i]
            #if statement includes approximated rgb ranges for possible shades of orange
            if r >= 200 and 110 <= g <= 150 and 10 <= b <= 60:
                x_ref, y_ref = j, i
                img_ref = get_ref_image(img, x_ref, y_ref)
                return x_ref, y_ref, img_ref, h
    
    return x_ref, y_ref, img, h

def get_ref_image(img, x_ref, y_ref):
    #creates green dot indication of reference frame on background image
    x_ref_upper = x_ref + 4
    y_ref_upper = y_ref + 4
    if x_ref < 3:
        x_ref_lower = 0
    else:
        x_ref_lower = x_ref - 3
    if y_ref < 3:
        y_ref_lower = 0
    else:
        y_ref_lower = y_ref - 3
    pix = img.load()
    for x in range(x_ref_lower, x_ref_upper):
        for y in range(y_ref_lower, y_ref_upper):
            pix[x,y] = (0, 255, 0)
    
    return img    

if __name__ == '__main__':
    img_path = 'C:/Users/IZ/Desktop/ISSAC/overhead/overhead/Multi_color/'
    csv_path = 'C:/Users/IZ/Desktop/ISSAC/overhead/overhead/groundtruth_coordinatesM.csv'
    background_path = 'C:/Users/IZ/Desktop/ISSAC/overhead/overhead/Multi_color/background.png'
    df = pandas.read_csv(csv_path, encoding = 'utf-8')
    X = []
    Y = []
    YAW = []
    x = 0
    y = 0
    yaw = 0
    counter = 0

    #get background 
    background = cv2.imread(background_path)
    background2 = background.copy()
    background2 = cv2.cvtColor(background,cv2.COLOR_BGR2GRAY)
    ret, background2 = cv2.threshold(background2, 240, 255, cv2.THRESH_TOZERO_INV)
    background2 = cv2.cvtColor(background2,cv2.COLOR_GRAY2BGR)
    
    #get reference coordinates for reference frame
    x_ref, y_ref, img_ref, h = get_ref_coordinates(background_path)
    #img_ref.show()
    
    #make thresh, make white transparent, overlay with original, track color (frameDelta)
    
    #iterate through all frames
    for i, row in tqdm(df.iterrows()):
        fname = row['file_location']
        #Time extracted from csv file
        ftime = row['time']
        print('iteration')
        interval = get_interval(ftime)
      
        #img is modified with boxes and added text
        img = cv2.imread(os.path.join(img_path, fname[:-1]))
        img_overlay = img.copy()

        #subtract background
        background2 = cv2.GaussianBlur(background2, (17, 17), 0)
        #img2 = cv2.GaussianBlur(img, (17, 17), 0)
        frameDelta = cv2.absdiff(background2, img)
        
        #frameDelta = cv2.GaussianBlur(frameDelta, (17, 17), 0)
        frameDelta = cv2.cvtColor(frameDelta,cv2.COLOR_BGR2RGB)
        frameDelta = cv2.GaussianBlur(frameDelta, (17, 17), 0)
        

        #track with framDelta
        x, y, yaw, img, v1_prev_velocity, v1_x_prev_ref, v1_y_prev_ref, img_overlay \
            = get_coordinates(img, interval, frameDelta, v1_prev_velocity, v1_x_prev_ref, v1_y_prev_ref, 
                              x, y, x_ref, y_ref, v1_lower_bound, v1_upper_bound, "RED", bgr_red, 150, img_overlay)
        
        x, y, yaw, img, v2_prev_velocity, v2_x_prev_ref, v2_y_prev_ref, img_overlay \
            = get_coordinates(img, interval, frameDelta, v2_prev_velocity, v2_x_prev_ref, v2_y_prev_ref, 
                              x, y, x_ref, y_ref, v2_lower_bound, v2_upper_bound, "BLUE", bgr_blue, 500, img_overlay)
        
        x, y, yaw, img, v3_prev_velocity, v3_x_prev_ref, v3_y_prev_ref, img_overlay \
            = get_coordinates(img, interval, frameDelta, v3_prev_velocity, v3_x_prev_ref, v3_y_prev_ref, 
                              x, y, x_ref, y_ref, v3_lower_bound, v3_upper_bound, "GREEN", bgr_green, 800, img_overlay)
        
        cv2.addWeighted(img_overlay, 0.2, img, 1, 0, img)

        # Resize to less than 640 and 480 pixel lengths respectively
        video_img = cv2.resize(img, (1280, 720), interpolation = cv2.INTER_AREA)
        video.write(video_img)
        
        
    video.release()
    cv2.destroyAllWindows()
    csv_input = pandas.read_csv(csv_path)
    #csv_input['Position_X'] = X
    #csv_input['Position_Y'] = Y
    #csv_input['Yaw'] = YAW
    #csv_input.to_csv('/home/jesse/Desktop/Logitech_Brio/output.csv', index=False)
    fraction = 0.3
    colors =  [plt.cm.rainbow(i) for i in np.linspace(0, 1, (counter))]
    ys = [i for i in range(int((counter-1)*fraction))]
    for y, c in zip(ys, colors):
        plt.scatter(X, Y, c=colors, s=2)
    print('Done')



0it [00:00, ?it/s]

iteration
34.420985
c
c
c
iteration
34.480820
c
c
c
c
c
c
iteration
34.540145
c
c
c
c
c
c
c
c


3it [00:00, 20.05it/s]

iteration
34.596489
c
c
c
c
c
c
c


4it [00:00, 19.95it/s]


Done
